## Overview

In `Nb.2.2ma` I had "[first structure error](https://discuss.ray.io/t/multi-agent-where-does-the-first-structure-comes-from/7010)" and I want to check what exactly was wrong by checking this [example](https://github.com/ray-project/ray/blob/master/rllib/examples/multi_agent_different_spaces_for_agents.py) :)

```
Example showing how one can create a multi-agent env, in which the different agents
have different observation and action spaces.
These spaces do NOT necessarily have to be specified manually by the user. Instead,
RLlib will try to automatically infer them from the env provided spaces dicts
(agentID -> obs/act space) and the policy mapping fn (mapping agent IDs to policy IDs).
---
Run this example with defaults (using Tune):
  $ python multi_agent_different_spaces_for_agents.py
```

### Boiler Plate

In [1]:
%reload_ext autoreload
%autoreload 2

## Import

In [2]:
import argparse
import gym
import os

import ray
from ray import tune #  air, 
from ray.rllib.env.multi_agent_env import MultiAgentEnv

C:\Users\milos\Anaconda3\envs\EnvRL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from ray.rllib.agents import ppo
from ray.tune.registry import register_env

## Environment

In [4]:
class BasicMultiAgentMultiSpaces(MultiAgentEnv):
    """A simple multi-agent example environment where agents have different spaces.
    agent0: obs=(10,), act=Discrete(2)
    agent1: obs=(20,), act=Discrete(3)
    The logic of the env doesn't really matter for this example. The point of this env
    is to show how one can use multi-agent envs, in which the different agents utilize
    different obs- and action spaces.
    """

    def __init__(self, config=None):
        self.agents = {"agent0", "agent1"}
        self._agent_ids = set(self.agents)

        self.dones = set()

        # Provide full (preferred format) observation- and action-spaces as Dicts
        # mapping agent IDs to the individual agents' spaces.
        self._spaces_in_preferred_format = True
        self.observation_space = gym.spaces.Dict(
            {
                "agent0": gym.spaces.Box(low=-1.0, high=1.0, shape=(10,)),
                "agent1": gym.spaces.Box(low=-1.0, high=1.0, shape=(20,)),
            }
        )
        self.action_space = gym.spaces.Dict(
            {"agent0": gym.spaces.Discrete(2), "agent1": gym.spaces.Discrete(3)}
        )

        super().__init__()

    def reset(self):
        self.dones = set()
        return {i: self.observation_space[i].sample() for i in self.agents}

    def step(self, action_dict):
        obs, rew, done, info = {}, {}, {}, {}
        for i, action in action_dict.items():
            obs[i] = self.observation_space[i].sample()
            rew[i] = 1.0
            done[i] = False
            info[i] = {}
        done["__all__"] = len(self.dones) == len(self.agents)
        return obs, rew, done, info

## RLlib

In [5]:
env_config = {}

In [6]:
config={
    "env": BasicMultiAgentMultiSpaces,
#     "env_config": env_config,
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
    "num_workers": 1,
    "multiagent": {
        # Use a simple set of policy IDs. Spaces for the individual policies
        # will be inferred automatically using reverse lookup via the
        # `policy_mapping_fn` and the env provided spaces for the different
        # agents. Alternatively, you could use:
        # policies: {main0: PolicySpec(...), main1: PolicySpec}
        "policies": {"main0", "main1"},
        # Simple mapping fn, mapping agent0 to main0 and agent1 to main1.
        "policy_mapping_fn": (
            lambda aid, episode, worker, **kw: f"main{aid[-1]}"
        ),
        # Only train main0.
        "policies_to_train": ["main0"],
    },
    "framework": "torch", # torch tf tf2
    "eager_tracing": "store_true",
}

In [7]:
agent_config = ppo.DEFAULT_CONFIG.copy()
agent_config.update(config)

## Trainig

In [8]:
# (Re)Start the ray runtime.
if ray.is_initialized():
    ray.shutdown()
ray.init(include_dashboard=False, ignore_reinit_error=True)

RayContext(dashboard_url=None, python_version='3.8.12', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:64403', 'raylet_socket_name': 'tcp://127.0.0.1:57948', 'webui_url': None, 'session_dir': 'C:\\Users\\milos\\AppData\\Local\\Temp\\ray\\session_2022-08-03_09-43-00_258476_15284', 'metrics_export_port': 56566, 'gcs_address': '127.0.0.1:57990', 'address': '127.0.0.1:57990', 'node_id': '605602b8041dae54fb0356d4e3ef35120be096c1fdd1e69b98f68f85'})

In [9]:
%%time

analysis = tune.run(
    ppo.PPOTrainer,
    stop={
        "training_iteration": 10,
        "timesteps_total": 10_000,
        "episode_reward_mean": 80.0,
    },
    config=agent_config,
    # Milos
    verbose = 0,    
    fail_fast = "raise", # for debugging!
)

C:\Users\milos\Anaconda3\envs\EnvRL\lib\site-packages\ray\tune\trial_runner.py:321: UserWarning: fail_fast='raise' detected. Be careful when using this mode as resources (such as Ray processes, file descriptors, and temporary files) may not be cleaned up properly. To use a safer mode, use fail_fast=True.
  warnings.warn(
2022-08-03 09:43:04,761	INFO trial_runner.py:803 -- starting PPOTrainer_BasicMultiAgentMultiSpaces_e890c_00000
2022-08-03 09:43:04,818	ERROR syncer.py:119 -- Log sync requires rsync to be installed.
(PPOTrainer pid=2020) 2022-08-03 09:43:09,263	INFO ppo.py:268 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=2020) 2022-08-03 09:43:09,264	INFO trainer.py:864 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=13316) 2022-08-03 09:43:13,216	WARNING rollout_worker.p

RayTaskError(ValueError): [36mray::PPOTrainer.train()[39m (pid=2020, ip=127.0.0.1, repr=PPOTrainer)
  File "python\ray\_raylet.pyx", line 663, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 667, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 614, in ray._raylet.execute_task.function_executor
  File "C:\Users\milos\Anaconda3\envs\EnvRL\lib\site-packages\ray\_private\function_manager.py", line 701, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\milos\Anaconda3\envs\EnvRL\lib\site-packages\ray\util\tracing\tracing_helper.py", line 462, in _resume_span
    return method(self, *_args, **_kwargs)
  File "C:\Users\milos\Anaconda3\envs\EnvRL\lib\site-packages\ray\tune\trainable.py", line 349, in train
    result = self.step()
  File "C:\Users\milos\Anaconda3\envs\EnvRL\lib\site-packages\ray\util\tracing\tracing_helper.py", line 462, in _resume_span
    return method(self, *_args, **_kwargs)
  File "C:\Users\milo